<a href="https://colab.research.google.com/github/woojung02/SSAC_AI/blob/main/VGGnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch  # PyTorch 핵심 모듈
import torch.nn as nn  # 신경망 레이어, 블록 등 제공
import torch.optim as optim  # 최적화 함수
import torchvision
import torchvision.transforms as transforms  # 이미지 전처리, 데이터 증강용
#VGGnet-16이용한 코드
# 1. device 설정 (GPU가 있으면 GPU 사용, 없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. CIFAR-10 데이터 전처리
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 입력 이미지를 무작위로 잘라서(패딩 포함) 과적합 방지(32*32를 패딩 하여 40*40으로 바꾸고 위리츷 무작위로 두고 다시 32*32로 자른다,이유:같은 이미지라도 다른 위치로 바꾸면 새로운 샘플처럼 활용 가능,또한 어떤 위치에 어떤 특징이 있다는것을 학습하는것을 막기 위해서 사용)
    transforms.RandomHorizontalFlip(),      # 무작위 좌우 반전으로 데이터 다양성 증가(좌우반전해서 같은 이미지라도 샘플 수를 늘릴수 있다.)
    transforms.ToTensor(),                  # 이미지를 텐서로 변환 (PyTorch 학습용)
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # 평균/표준편차로 정규화 (학습 안정, 속도 증가)
                         (0.2023, 0.1994, 0.2010))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

# 3. CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 4. VGG Block 구현 함수
def vgg_block(num_convs, in_channels, out_channels):
    # num_convs : 연속 합성곱 레이어 수, in_channels : 입력 채널, out_channels : 출력 채널
    layers = []
    for i in range(num_convs):
        # Conv2d: 합성곱 연산, kernel_size=3, padding=1(출력 크기 보존), bias=False(BatchNorm과 궁합)
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False))
        layers.append(nn.BatchNorm2d(out_channels))  # BatchNorm: 학습 안정성, 수렴 속도 개선
        layers.append(nn.ReLU(inplace=True))         # ReLU: 비선형성, 활성화 함수
        in_channels = out_channels                   # 다음 conv의 입력 채널을 현재 출력 채널로
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))  # MaxPool: 특징 맵 크기 절반으로 줄임(공간 정보 압축)
    return nn.Sequential(*layers)                    # 여러 레이어를 묶어서 하나의 블록으로 반환

# 5. VGGNet 전체 네트워크 구현
class VGGNet(nn.Module):
    def __init__(self, num_classes=10):
        super(VGGNet, self).__init__()
        # CIFAR-10은 (3,32,32) 이미지 → kernel=3, padding=1로 크기 유지

        # VGG-16 레이어 구조: conv개수, 입력/출력 채널 수를 리스트로 관리
        conv_arch = [
            (2, 3, 64),      # conv 2개, in 3, out 64
            (2, 64, 128),    # conv 2개, in 64, out 128
            (3, 128, 256),   # conv 3개, in 128, out 256
            (3, 256, 512),   # conv 3개, in 256, out 512
            (3, 512, 512),   # conv 3개, in 512, out 512
        ]

        # 합성곱 블록 모듈 쌓기
        layers = []
        for (num_convs, in_c, out_c) in conv_arch:
            layers.append(vgg_block(num_convs, in_c, out_c))
        self.features = nn.Sequential(*layers)   # 합성곱 계층 전체를 하나로 묶음

        # FC(fully connected) 부분: 특징맵을 일렬로 펼쳐 분류
        # CIFAR-10 기준, 마지막 맵 크기는 1x1이 아닌 1x1이 아닐 수 있으니 AdaptiveAvgPool 사용
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # (채널수, 1, 1)로 강제 변환
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),  # 512(채널) x 1 x 1 = 512 → 4096
            nn.ReLU(True),
            nn.Dropout(0.5),       # 과적합 방지 드롭아웃
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes), # 최종 클래스 개수만큼 출력(10)
        )

    def forward(self, x):
        x = self.features(x)  # 합성곱(특징추출) 블록 통과
        x = self.avgpool(x)   # (B, 512, H, W) → (B, 512, 1, 1)
        x = torch.flatten(x, 1)  # (B, 512, 1, 1) → (B, 512)
        x = self.classifier(x)   # FC 계층 통과해 분류 결과 출력
        return x

# 6. 모델 인스턴스 생성, device로 이동
model = VGGNet(num_classes=10).to(device)

# 7. 손실함수, 옵티마이저, 스케줄러 정의(스케줄러 이용해서 최적의 learning late을 찾기 쉬워진다)
criterion = nn.CrossEntropyLoss()   # 다중 클래스 분류용 손실함수
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4) # SGD 옵티마이저
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40, 70], gamma=0.1)  # 학습률 스케줄러

# 8. 학습 함수
def train(epoch):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()                # 이전 배치의 기울기 리셋
        outputs = model(inputs)              # 순전파
        loss = criterion(outputs, targets)   # 손실 계산
        loss.backward()                     # 역전파(기울기 계산)
        optimizer.step()                    # 파라미터 업데이트
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 100 == 99:
            print(f"[{epoch}][{batch_idx+1}] Loss: {running_loss/100:.3f}, Acc: {100.*correct/total:.2f}%")
            running_loss = 0.0

# 9. 테스트 함수
def test(epoch):
    model.eval()
    correct, total = 0, 0
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print(f"==> Epoch {epoch} Test Loss: {test_loss/len(testloader):.3f}, Acc: {100.*correct/total:.2f}%")

# 10. 전체 학습 루프
num_epochs = 100
for epoch in range(1, num_epochs+1):
    train(epoch)
    test(epoch)
    scheduler.step()  # 학습률 감소 스케줄 적용


100%|██████████| 170M/170M [00:05<00:00, 29.2MB/s]


[1][100] Loss: 2.135, Acc: 17.96%
[1][200] Loss: 1.883, Acc: 22.20%
[1][300] Loss: 1.726, Acc: 25.32%
==> Epoch 1 Test Loss: 1.570, Acc: 41.55%
[2][100] Loss: 1.547, Acc: 41.34%
[2][200] Loss: 1.444, Acc: 43.77%
[2][300] Loss: 1.383, Acc: 45.89%
==> Epoch 2 Test Loss: 1.290, Acc: 53.18%
[3][100] Loss: 1.235, Acc: 56.34%
[3][200] Loss: 1.166, Acc: 57.79%
[3][300] Loss: 1.073, Acc: 59.47%
==> Epoch 3 Test Loss: 1.044, Acc: 64.54%
[4][100] Loss: 0.995, Acc: 65.79%
[4][200] Loss: 0.935, Acc: 66.91%
[4][300] Loss: 0.890, Acc: 67.95%
==> Epoch 4 Test Loss: 1.215, Acc: 62.86%
[5][100] Loss: 0.854, Acc: 71.64%
[5][200] Loss: 0.814, Acc: 72.31%
[5][300] Loss: 0.786, Acc: 72.71%
==> Epoch 5 Test Loss: 1.258, Acc: 62.83%
[6][100] Loss: 0.738, Acc: 76.30%
[6][200] Loss: 0.692, Acc: 76.89%
[6][300] Loss: 0.705, Acc: 76.92%
==> Epoch 6 Test Loss: 0.666, Acc: 77.82%
[7][100] Loss: 0.663, Acc: 78.41%
[7][200] Loss: 0.666, Acc: 78.49%
[7][300] Loss: 0.659, Acc: 78.52%
==> Epoch 7 Test Loss: 0.759, Acc:

In [ ]:
import torch  # PyTorch 핵심 모듈
import torch.nn as nn  # 신경망 레이어, 손실 함수 등
import torch.optim as optim  # 최적화 알고리즘
import torchvision  # 컴퓨터 비전용 데이터셋, 모델
import torchvision.transforms as transforms  # 이미지 전처리, 증강
#VGGnet 요약(토치비션에서 불러와서 사용하기)
# 1. 장치 설정: GPU가 있으면 'cuda', 없으면 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. CIFAR-10 학습용 전처리 + 증강
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 32×32에 4px 패딩→40×40→랜덤 크롭
    transforms.RandomHorizontalFlip(),      # 좌우 반전 증강
    transforms.ToTensor(),                  # [0,255]→[0,1] 텐서 변환
    transforms.Normalize((0.4914,0.4822,0.4465),  # 채널별 평균 정규화
                         (0.2023,0.1994,0.2010))  # 채널별 표준편차 정규화
])

# 3. CIFAR-10 테스트용 전처리
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2023,0.1994,0.2010))
])

# 4. 데이터셋 로드 및 DataLoader 생성
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# 5. torchvision의 VGG-16 불러오기
from torchvision.models import vgg16

model = vgg16(pretrained=False)             # 사전학습된 가중치 없이 초기화
model.classifier[-1] = nn.Linear(4096, 10)   # 마지막 FC를 CIFAR-10(10클래스)에 맞게 교체
model = model.to(device)                    # 모델을 GPU/CPU로 이동

# 6. 손실함수와 옵티마이저, 스케줄러 정의
criterion = nn.CrossEntropyLoss()            # 분류용 손실
optimizer = optim.SGD(model.parameters(), lr=0.05,  # 학습률 0.05
                      momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                           milestones=[40, 70], gamma=0.1)

# 7.train함수 정의
def train(epoch):
    model.train()  # 학습 모드 활성화 (드롭아웃·배치정규화 작동)
    running_loss = 0.0
    correct = total = 0
    print(f"Epoch {epoch} 시작")
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)  # GPU로 이동
        optimizer.zero_grad()             # 이전 기울기 초기화
        outputs = model(inputs)           # 순전파
        loss = criterion(outputs, targets)# 손실 계산
        loss.backward()                   # 역전파
        optimizer.step()                  # 파라미터 업데이트

        running_loss += loss.item()
        _, predicted = outputs.max(1)     # 가장 높은 확률 클래스 선택
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # 100배치마다 로그
        if batch_idx % 100 == 99:
            print(f"  Batch {batch_idx+1}, "
                  f"Loss: {running_loss/100:.3f}, "
                  f"Acc: {100.*correct/total:.2f}%")
            running_loss = 0.0

# 8. 테스트
def test(epoch):
    model.eval()  # 평가 모드 활성화 (드롭아웃·배치정규화 비활성화)
    test_loss = correct = total = 0
    with torch.no_grad():  # 기울기 계산 비활성화
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f"==> Epoch {epoch} Test Loss: {test_loss/len(testloader):.3f}, "
          f"Acc: {100.*correct/total:.2f}%")

# 9. 전체 학습 루프
num_epochs = 100
for epoch in range(1, num_epochs+1):
    train(epoch)      # 학습
    test(epoch)       # 평가
    scheduler.step()  # 학습률 스케줄러 갱신


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1 시작
  Batch 100, Loss: 2.277, Acc: 13.09%
  Batch 200, Loss: 2.228, Acc: 13.93%
  Batch 300, Loss: 2.117, Acc: 15.33%
==> Epoch 1 Test Loss: 1.993, Acc: 18.06%
Epoch 2 시작
  Batch 100, Loss: 1.954, Acc: 20.88%
  Batch 200, Loss: 1.864, Acc: 22.84%
  Batch 300, Loss: 1.817, Acc: 24.49%
==> Epoch 2 Test Loss: 1.822, Acc: 31.13%
Epoch 3 시작
  Batch 100, Loss: 1.753, Acc: 31.80%
  Batch 200, Loss: 1.720, Acc: 32.50%
  Batch 300, Loss: 1.628, Acc: 33.92%
==> Epoch 3 Test Loss: 1.554, Acc: 38.18%
Epoch 4 시작
  Batch 100, Loss: 1.531, Acc: 41.72%
  Batch 200, Loss: 1.519, Acc: 42.62%
  Batch 300, Loss: 1.518, Acc: 42.97%
==> Epoch 4 Test Loss: 1.252, Acc: 54.26%
Epoch 5 시작
  Batch 100, Loss: 1.386, Acc: 50.63%
  Batch 200, Loss: 1.339, Acc: 51.75%
  Batch 300, Loss: 1.273, Acc: 53.24%
==> Epoch 5 Test Loss: 1.201, Acc: 58.16%
Epoch 6 시작
  Batch 100, Loss: 1.199, Acc: 58.71%
  Batch 200, Loss: 1.119, Acc: 60.40%
  Batch 300, Loss: 1.128, Acc: 60.77%
==> Epoch 6 Test Loss: 1.010, Acc: 65.55